<a href="https://colab.research.google.com/github/apchavezr/16.-Aprendizaje-Profundo-para-Ciencia-de-Datos/blob/main/cnn_plantvillage_clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de enfermedades de plantas con CNN y PlantVillage

Este cuaderno guía al estudiante en la implementación paso a paso de una red neuronal convolucional (CNN) para clasificar enfermedades de plantas usando el conjunto de datos PlantVillage. También se incluyen visualizaciones de activaciones y métricas de evaluación como la matriz de confusión y las curvas ROC.

Este ejercicio permite a los estudiantes adquirir experiencia en tareas prácticas de clasificación de imágenes, visualización de activaciones internas y evaluación del rendimiento de modelos de aprendizaje profundo aplicados a problemas reales del sector agrícola.

🎯 **Objetivo del ejercicio**

Entrenar y evaluar una CNN capaz de clasificar imágenes de hojas de plantas sanas y enfermas del conjunto PlantVillage. El estudiante observará cómo la red aprende patrones visuales, analizará las activaciones internas y evaluará métricas como la matriz de confusión y la curva ROC.

🔧 Paso 1. Preparación del entorno

In [1]:
# Instalación de librerías necesarias
!pip install -q tensorflow scikit-learn matplotlib seaborn opencv-python

📥 Paso 2. Carga y preprocesamiento de los datos

**a)** Cargar imágenes desde un repositorio de Kaggle o Google Drive

Se recomienda usar un subconjunto del dataset PlantVillage en Kaggle, o subir las carpetas directamente a Colab.

In [3]:
import kagglehub

# Descargar la última versión
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/plantdisease


In [6]:
# Carga de datos desde directorios
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 128
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    '/kaggle/input/plantdisease',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/kaggle/input/plantdisease',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 33022 images belonging to 2 classes.
Found 8254 images belonging to 2 classes.


🧠 Paso 3. Construcción y entrenamiento del modelo CNN

In [7]:
# Construcción de la red neuronal convolucional
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Entrenamiento del modelo
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5010 - loss: 0.7115

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1260s 1s/step - accuracy: 0.5010 - loss: 0.7115 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1219s 1s/step - accuracy: 0.5032 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
 319/1032 ━━━━━━━━━━━━━━━━━━━━ 12:46 1s/step - accuracy: 0.4949 - loss: 0.6932

KeyboardInterrupt: 

🔍 Paso 4. Visualización de activaciones

Esto permite observar qué “ve” la red en sus primeras capas.

In [ ]:
# Visualización de activaciones
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import Model

img, _ = val_gen.next()
sample_img = img[0]

layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(sample_img[np.newaxis, ...])

fig, axes = plt.subplots(4, 8, figsize=(12,6))
for i, ax in enumerate(axes.flat):
    ax.imshow(activations[0][0, :, :, i], cmap='viridis')
    ax.axis('off')
plt.suptitle('Activaciones de la primera capa')
plt.tight_layout()
plt.show()

📊 Paso 5. Evaluación del rendimiento

**a) **Matriz de confusión

In [ ]:
# Matriz de confusión
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(val_gen)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_gen.classes

cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=val_gen.class_indices.keys())
disp.plot(xticks_rotation=90)
plt.title('Matriz de confusión')
plt.show()

**b) **Curvas ROC para múltiples clases

In [ ]:
# Curvas ROC por clase
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

y_test_bin = label_binarize(y_true, classes=range(train_gen.num_classes))
fpr, tpr, roc_auc = {}, {}, {}

for i in range(train_gen.num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i in range(train_gen.num_classes):
    plt.plot(fpr[i], tpr[i], label=f"Clase {i} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title('Curvas ROC por clase')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.grid()
plt.show()

✅** Conclusión**

Este ejercicio permite:

   *    Entrenar una CNN en imágenes reales de plantas afectadas por enfermedades.

   *  Visualizar cómo las capas convolucionales identifican patrones visuales.

   *  Evaluar objetivamente la capacidad del modelo para generalizar sobre nuevas imágenes.

Este tipo de experiencias contribuye al desarrollo de habilidades prácticas en visión por computador aplicada a contextos agrícolas, útiles en desarrollos de soluciones inteligentes en el sector agropecuario colombiano.